In [9]:
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from agent import *
from selection import *
from utils import *
from env import Environment
from simulation import constructAgent
from main import Config

In [10]:
MULTI_SELECTION_METHOD = 'LSTM-VAR'
config = {
    'reward': 3, 
    'sucker': 0, 
    'temptation': 5, 
    'punishment': 1, 
    'n_episodes': 2000, 
    'discount': 0.95,
    'play_epsilon': 1,
    'select_epsilon': 1,
    'epsilon_decay': 0.99,
    'min_epsilon': 0.01,
    'alpha': 0.1,
    'n_actions': 2,
    'h': 10,
    'state_repr': 'bi-repr',
    'batch_size': 64,
    'learning_rate': 1e-3,
}
config = Config(config)
env = Environment(config)
seed_everything()

In [20]:
agents = {}
index = 0
with HiddenPrints():
    # for _ in range(5):
        # agents[index] = constructAgent('TitForTat', config)
        # index += 1
    for _ in range(10):
        agents[index] = constructAgent('LSTMQN', config)
        index += 1
 
agents = lstm_variant_selection(config, agents, env)

for n in range(len(agents)):
    print('Agent{}: name:{}  final score:{}  play time:{}  times to play D:{}  ratio: {}  faced D ratio: {}'
        .format(n, agents[n].name, agents[n].running_score,
        len(agents[n].own_memory[:agents[n].play_times]), list(agents[n].own_memory[:agents[n].play_times]).count(1),
                list(agents[n].own_memory[:agents[n].play_times]).count(1)/len(agents[n].own_memory[:agents[n].play_times]),
                list(agents[n].oppo_memory[:agents[n].play_times]).count(1)/len(agents[n].oppo_memory[:agents[n].play_times])))
print('The reward for total society: {}'.format(env.running_score/len(agents)))

100%|██████████| 2000/2000 [24:52<00:00,  1.34it/s]


Agent0: name:LSTMQN  final score:51.61698601267451  play time:6579  times to play D:1005  ratio: 0.15275877792977657
Agent1: name:LSTMQN  final score:42.19863642339305  play time:4397  times to play D:66  ratio: 0.015010234250625426
Agent2: name:LSTMQN  final score:98.12268555809025  play time:2668  times to play D:2590  ratio: 0.9707646176911544
Agent3: name:LSTMQN  final score:85.15378434291158  play time:2086  times to play D:2026  ratio: 0.9712368168744008
Agent4: name:LSTMQN  final score:96.53696690247916  play time:2477  times to play D:2424  ratio: 0.9786031489705289
Agent5: name:LSTMQN  final score:85.02517929406315  play time:2061  times to play D:897  ratio: 0.43522561863173215
Agent6: name:LSTMQN  final score:84.24989252494744  play time:2361  times to play D:2294  ratio: 0.9716221939855993
Agent7: name:LSTMQN  final score:35.77322283832727  play time:9048  times to play D:95  ratio: 0.010499557913351016
Agent8: name:LSTMQN  final score:51.282586767376905  play time:4585  ti

In [21]:
def get_index_from_action(action, idx):
    scale = lambda x: x+1 if x>=idx else x
    return list(map(scale, action))

for idx in agents:
    transitions = agents[idx].SelectMemory.memory
    _, actions, rewards, _ = zip(*transitions)
    actions = get_index_from_action(np.array(actions, dtype=int), idx)
    actions, rewards = np.array(actions), np.array(rewards)
    print(f'Agent {idx}:', end='')
    values, counts = np.unique(actions, return_counts=True)
    print(f' opponent_idx: {list(values)}, counts: {list(counts)} ', end='')
    dict_idx = {x: rewards[np.where(actions == x)] for x in values}
    print(f'rewards: {[np.mean(y) for _, y in dict_idx.items()]}')
    # print(f'rewards: {[np.std(y) for _, y in dict_idx.items()]}')
# plt.plot(actions)
# plt.show()
# print(rewards)

Agent 0: opponent_idx: [3, 4, 7, 8, 9], counts: [3, 6, 880, 1, 110] rewards: [0.6666666666666666, 0.8333333333333334, 3.3420454545454548, 3.0, 2.9727272727272727]
Agent 1: opponent_idx: [0, 2, 3, 5, 6, 7, 8, 9], counts: [1, 21, 33, 1, 11, 881, 2, 50] rewards: [3.0, 0.0, 0.0, 0.0, 0.0, 2.989784335981839, 3.0, 3.08]
Agent 2: opponent_idx: [1, 3, 5, 6, 7, 8], counts: [433, 1, 4, 1, 1, 560] rewards: [4.963048498845265, 1.0, 1.0, 0.0, 5.0, 4.957142857142857]
Agent 3: opponent_idx: [0, 1, 2, 5, 6, 7, 8, 9], counts: [62, 342, 3, 30, 1, 63, 1, 498] rewards: [3.5806451612903225, 4.970760233918129, 1.0, 1.0, 1.0, 4.873015873015873, 5.0, 4.963855421686747]
Agent 4: opponent_idx: [0, 2, 3, 5, 6, 7, 9], counts: [1, 2, 4, 1, 2, 988, 2] rewards: [1.0, 1.0, 1.0, 1.0, 1.0, 4.9574898785425106, 5.0]
Agent 5: opponent_idx: [0, 1, 2, 6, 7, 8, 9], counts: [3, 134, 26, 1, 436, 377, 23] rewards: [4.333333333333333, 4.343283582089552, 1.0, 1.0, 3.7362385321100917, 3.7586206896551726, 4.913043478260869]
Agent 6